# Part I. ETL Pipeline for Pre-Processing the Files

## Files Preprocessing

#### Import Python packages 

In [24]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [25]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [26]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
#print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [27]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [28]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [29]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor': 1 }"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [30]:
try:
    session.set_keyspace("project1")
except Exception as e:
    print(e)

## Part II. Data Modeling with Apache Cassandra

### We are going to create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'



### Question 1:
Question 1 asks the following: given the sessionId and itemInSession, we need the artists, titles of the songs and lengths of the songs that were played during that session.

#### Query design
Based on the question description we can formulate the query as follows:

```SELECT artist, song_title, song_length FROM TABLE_NAME WHERE session_id = value AND item_in_session = value```

#### Creating the table
Since the query is asking about songs in a specific session we are going to name the table music_session.
Furthmore, because we are conditioning of sessionId and itemInSession and they uniquely identify a single row, they are going to be the primary key.


In [31]:
query1 = "SELECT artist, song_title, song_length FROM music_session WHERE session_id=338 and item_in_session=4"
create_table_1 = "CREATE TABLE IF NOT EXISTS music_session "
create_table_1 += "(session_id int, item_in_session int, artist text, song_title text, song_length float, \
    PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(create_table_1)
except Exception as e:
    print(e)

In [32]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_session (session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [33]:
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print(f"artist: {row[0]}, song title: {row[1]}, song length: {row[2]}")

artist: Faithless, song title: Music Matters (Mark Knight Dub), song length: 495.30731201171875


### Question 2:
Question 2 asks the following: given the userId and sessionId, we need the user full name, the artists and titles of the songs the user listened to in that session. Furthmore, the songs should be sorted by itemInSession.

#### Query design
Based on the question description we can formulate the query as follows:

```SELECT artist, song_title, user_fname, user_lname FROM TABLE_NAME WHERE user_id = value and session_id = value```

#### Creating the table
Since the query is asking about songs in a specific session for a specific user, we are going to name the table user_session_history.
Furthmore, because we are conditioning of userId and sessionId and we need to sort by itemInSession, then these three attributes are going to be the primary key:

- userId and sessionId as the partition key since we need data for one user session to be in one partition.
- itemInSession as a clustering column for sorting.

In [34]:
query2 = "SELECT artist, song_title, user_fname, user_lname FROM user_session_history WHERE user_id=10 and session_id=182"
create_table_2 = "CREATE TABLE IF NOT EXISTS user_session_history "
create_table_2 += "(user_id int, session_id int, item_in_session int, artist text, song_title text, user_fname text, \
    user_lname text, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(create_table_2)
except Exception as e:
    print(e)    

In [35]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_history (user_id, session_id, item_in_session, artist, song_title, \
            user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [37]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print(f"artist: {row[0]}, song title: {row[1]}, user name: {row[2] + ' ' + row[3]}")

artist: Down To The Bone, song title: Keep On Keepin' On, user name: Sylvie Cruz
artist: Three Drives, song title: Greece 2000, user name: Sylvie Cruz
artist: Sebastien Tellier, song title: Kilometer, user name: Sylvie Cruz
artist: Lonnie Gordon, song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), user name: Sylvie Cruz


### Question 3:
Question 3 asks the following: given a specific song title, we need all users' full name who listened to that song.

#### Query design
Based on the question description we can formulate the query as follows:

```SELECT user_fname, user_lname FROM TABLE_NAME WHERE song_title = value```

#### Creating the table
Since the query is asking about the users who listened to a specific songs, we are going to name the table song_play_history.
Furthmore, since song_title does not uniquely identifies a row, and user full name can have duplicate values, we are going to use both song_title and user_id as the primary key.

In [38]:
query3 = "SELECT user_fname, user_lname FROM song_play_history WHERE song_title='All Hands Against His Own'"
create_table_3 = "CREATE TABLE IF NOT EXISTS song_play_history "
create_table_3 += "(song_title text, user_id int, user_fname text, user_lname text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(create_table_3)
except Exception as e:
    print(e)             

In [39]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_play_history (song_title, user_id, user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [40]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(f"user name: {row[0] + ' ' + row[1]}")

user name: Jacqueline Lynch
user name: Tegan Levine
user name: Sara Johnson


### Dropping the tables before closing out the sessions

In [41]:
query = "drop table music_session"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
    
query = "drop table user_session_history"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_play_history"
try:
    session.execute(query)
except Exception as e:
    print(e)


### Closing the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()